In [31]:
# import the required modules
import sqlalchemy as db

In [32]:
# Check the version of the sqlalchemy
db.__version__

'1.4.32'

In [3]:
# install from jupyter notebook if any supported packages are required
# In our case, we need the psycopg2 driver to be able to connect to the postgresDB
!python -m pip install psycopg2-binary 

### Establish connection:

Set the credentials that you have created during the installation or setup of your DB (postgresDB)

In [34]:
# credentials for connecting to your DB
user = 'postgres'
password = '123456'
host = 'localhost'
port = '5432'
dbn = 'postgres'

Create a connection to the DB

In [35]:
connect = 'postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbn)
engine = db.create_engine(connect)
connection = engine.connect()

### Initialize the Metadata Object

Create an metadata object to access the DB

In [36]:
metadata = db.MetaData()

### Create a table schema

In [37]:
emp = db.Table('emp', metadata,
              db.Column('Id', db.Integer()),
              db.Column('name', db.String(255), nullable=False),
              db.Column('salary', db.Float(), default=100.0),
              db.Column('active', db.Boolean(), default=True))

### Create the table

In [38]:
metadata.create_all(engine) # creates the table

In [39]:
emp.create(engine, checkfirst=True) # other method

### View all the tables in the DB using metadata object

In [40]:
for t in metadata.sorted_tables:
    print(t.name)

emp


Feel free to add more parameters to the print statement to view more about the table

### View Schema of a table:

In [41]:
for c in emp.c:
    print(c, c.type, c.nullable)

emp.Id INTEGER True
emp.name VARCHAR(255) False
emp.salary FLOAT True
emp.active BOOLEAN True


### View Table

In [42]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

In [43]:
Session = sessionmaker(bind=engine)
session = Session()

#### Method-1

In [44]:
stmt = select('*').select_from(emp)
result = session.execute(stmt).fetchall()
result

[]

#### Method-2

In [45]:
sql = "select * from emp;"
connection.execute(sql).fetchall()

[]

### Insert Data:

In [16]:
query = db.insert(emp).values(Id=2, name='Harry', salary=10000.00, active=True) 
ResultProxy = connection.execute(query)

In [17]:
query = db.insert(emp) 
values_list = [{'Id':'9', 'name':'Richard', 'salary':50000, 'active':False},
               {'Id':'5', 'name':'Howard', 'salary':20000, 'active':True}]
ResultProxy = connection.execute(query, values_list)

To view all the data

In [18]:
sql = "select * from emp;"
connection.execute(sql).fetchall()

[(2, 'Harry', 10000.0, True),
 (9, 'Richard', 50000.0, False),
 (5, 'Howard', 20000.0, True)]

### Update:

In [19]:
from sqlalchemy import update

In [20]:
query = update(emp).values({"name":"Nick"}).where(emp.c.Id == '5')
ResultProxy = connection.execute(query)

In [21]:
stmt = select('*').select_from(emp)
result = session.execute(stmt).fetchall()
result

[(2, 'Harry', 10000.0, True),
 (9, 'Richard', 50000.0, False),
 (5, 'Nick', 20000.0, True)]

### Delete

Truncate a table - remove data without deleting the table itself

In [22]:
query = emp.delete()
result = connection.execute(query)

In [23]:
stmt = select('*').select_from(emp)
result = session.execute(stmt).fetchall()
result

[]

### Drop:

Drop the table itself. 

In [24]:
session.close() # this is important. Close all the open sessions to be able to drop a table

In [25]:
emp.drop(engine) #drops a single table

In [29]:
metadata = db.MetaData()
for t in metadata.sorted_tables:
    print(t.name)

Nothing returned. The table has been deleted.

### Drop all the tables:

In [26]:
metadata.drop_all(engine) #drops all the tables in the database

In [30]:
metadata = db.MetaData()
for t in metadata.sorted_tables:
    print(t.name)

Nothing returned. The table has been deleted.

---